# Summer Project 2017
## Blowfly strike in rabbits

### Import the required libraries
(You will need to make sure that you pip install epydemiology package into your virtual Python environment before proceeding.)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pymysql
import getpass
import re, textwrap
import socket
import collections
import epydemiology as epy
import matplotlib
# import matplotlib as mpl
# import matplotlib.pyplot as plt

### Define the function that will download the data

In [ ]:
def phjGetSAVSNETData(phjQuery,
                      phjPrintResults = True):
    
    # Make connection to MySQL database
    # =================================
    phjMaxAttempts = 3    # Max number of attmepts to make connection to database
    
    for i in range(phjMaxAttempts):
        # phjServer = input("Enter host: ")
        # phjServer = '138.253.152.243'
        phjServer = 'savsnetnlp.vets.liv.ac.uk'
        
        phjUser = input("Enter MySQL database username: ")
        phjPwd = getpass.getpass("Enter password: ")
        
        try:
            phjConnection = pymysql.connect(host=phjServer, user=phjUser,password=phjPwd)
            break    # Break to leap out of for loop
            
        except pymysql.err.OperationalError as e:
            print("\nAn OperationalError occurred. Error number {0}: {1}.".format(e.args[0],e.args[1]))
            
            if i < (phjMaxAttempts-1):
                print("\nPlease try again.\n")    # Don't say 'Please try again' if last attempt has failed.
            
        except socket.timeout:
            print("Timed out")
    
    # Report that connection failed after maximum number of attempts and return None
    if i == (phjMaxAttempts-1):
        print("\nFailed to make connection after {0} attempts.".format(i+1))
        return None
    
    # If connection has been made successfully...
    else:
        if phjPrintResults:
            print("\nQuery used to interrogate database =\n",phjQuery,"\n")
        
        phjTempDF = pd.io.sql.read_sql(phjQuery, con=phjConnection)
                    
        phjConnection.close()

        if phjPrintResults:
            print('\n')
            print('Query completed')

            print('Number of rows returned = ',len(phjTempDF.index))

        return phjTempDF

### Define the SQL query that will request the required data

In [ ]:
# This query will download the complete Robovet narrative data. However, is has
# been noted that the records included in the deidentified narrative table and
# the table containing additional data are different. This is because the queries
# used to extract each dataset were different. There was also an issue were
# the query used to generate the table containing additional data did not include
# many of the early records.
# Consequently, to ensure the data read by students matches records that are
# available to get additional information, the following query can be used.
phjQuery = '''SELECT DISTINCT `RobovetConsultations_2017-05-19`.*,
                              `deidentifiedRobovet`.`narrative` AS 'deidentifiedNarrative'
              FROM `robovetDeidentified2017-01-30`.`RobovetConsultations_2017-05-19`
                  LEFT JOIN `robovetDeidentified2017-01-30`.`deidentifiedRobovet`
                      ON `RobovetConsultations_2017-05-19`.`consultID` = `deidentifiedRobovet`.`consultID`
              WHERE (`deidentifiedRobovet`.`narrative` IS NOT NULL AND 
                     `deidentifiedRobovet`.`narrative` != "" AND
                     `deidentifiedRobovet`.`narrative` != "\"\"")
                  AND
                    `RobovetConsultations_2017-05-19`.`species` = 'rabbit'
            '''

print(phjQuery)

### Load the required data into a Pandas dataframe

In [ ]:
df = phjGetSAVSNETData(phjQuery, phjPrintResults = True)

### Print out examples of the dataframe

In [ ]:
print(df.columns)
print('\n')

with pd.option_context('display.max_rows', 10, 'display.max_columns', 10):
    print(df)

### Compile the final version of the regex
(N.B. There were a few typos in the original version (/s rather than \s to indicate a space) which have been corrected.)

In [ ]:
theFinalRegex = re.compile( r"""(?P<RabbitFS>
                                (?<!disc.)
                                (?<!cuss.)
                                (?<!sion.)
                                (?<!ssed.)
                                (?<!vise.)
                                (?<!rned.)   # Removes any variant of discussed, advise, warned, prevent, risk, about, of, from
                                (?<!vent.)
                                (?<!risk.)
                                (?<!bout.)
                                (?<!..of.)
                                (?<!from.)
                                (?<!s/s/no.)   # Removes no, and, re, for, avoid, diet, dietary, food, vaccination, 
                                (?<!.and.)
                                (?<!..re.)
                                (?<!.for.)
                                (?<!void.)
                                (?<!iet..)
                                (?!<ing..)   # Removes any variant identified in the output data that was a false positive
                                (?<!ant..)
                                (?<!dary..)
                                (?<!ood..)
                                (?<!are..)
                                (?<!vacc..)   
                                (?<!.ion.)
                                (?<!ase..)
                                (?<!an..)
                                (?<!nce..)
                                (?<!rol..)   
                                (?<!inst.)
                                (?<!ware.)
                                (?:fly.*strike|mag+ot+|my?ia?sis)
                                (?!\scontrol)(?!\sprevent)(?!\sprotect)) #Removes flys strike that has been followed by control, prevent, protect
                             """, flags=re.I|re.X)

### Tag the consultations where the regex finds a match
**A new column called 'foundit' will be added to the dataframe and will contain either True or False depending on whether your regex found a match.**

In [ ]:
print(df.columns)
print('\n')

if 'foundit' in df.columns:
    df = df.drop('foundit',axis = 1)

df['foundit'] =  df['deidentifiedNarrative'].str.contains(theFinalRegex)
df['founditBinary'] = df['foundit'].astype(int)

with pd.option_context('display.max_rows', 20, 'display.max_columns', 10):
    print(df.loc[df['foundit']==True,:])

In [ ]:
print(df['consultMonth'].unique())

### Create a series of new columns in the database, one for each capturing group in the regex.
(Since there is only one group in the rabbit regex, this column will be the same as the 'foundit' column.)

In [ ]:
# Extract all matching strings from the consultations
dfMatches =  df['deidentifiedNarrative'].str.extractall(theFinalRegex)

# Define an empty dataframe
dfLists = pd.DataFrame()

# Concatenate multiple matches (over several rows) into a single list (on one row)
for column in dfMatches:
    dfLists[column] = dfMatches.groupby(level=0)[column].apply(list)

# Remove columns if already exist
columnNamesList = list(dfLists.columns.values)

for col in columnNamesList:
    if col in df.columns:
        df = df.drop(col,axis = 1)

df = df.join(dfLists,how="left")

In [ ]:
# Create a variable that shows number of weeks since start
# --------------------------------------------------------
# The consultWeek variable in the dataframe gives the week number in the year and returns to 1
# at the beginning of each year. It also produces weeks that contain less tha 7 days at the
# start and end of the year. Is there also a possibility that a year may contain 53 weeks?
# Instead, to produce a variable that represents the number of weeks since a specific date
# then calculate the number of days since the start followed by division by 7, stored as an int.

df['consultDate'] = pd.to_datetime(df['consultDate'])

phjStartDate = pd.to_datetime('2014-01-06')

print(phjStartDate)

df['days_since_start'] = (df['consultDate'] - phjStartDate).dt.days
df['weeks_since_start'] = df['days_since_start'].divide(7).astype(int)

In [ ]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', 10):
    print(df)

### Label consultations that have already been read and confirmed to be positive

In [ ]:
phjPrevReadConsultations = [114546,83451,85847,89505,2048740,93728,94355,149066,95415,99034,102413,104942,106778,108215,109743,450658,112293,120154,120916,123916,123926,124486,128396,136553,133457,135799,136903,159938,148063,153611,157777,160917,165891,165898,179656,184644,181223,193588,203170,232673,253041,425367,465311,390105,1932466,465708,521808,451686,469037,488289,2164286,511155,513561,514301,516756,525640,530185,534479,2048805,1675990,600421,602639,1795734,624409,646856,646930,653038,672228,1827616,1839680,2166400,2033838,2126000,2165245,2216207,1806567,908371,2168489,1874908,2063876,1718220,1947558,1286882,1932054,1992851,1789812,2168025,2317312,1611674,1628998,1630982,1635634,1637712,1861375,1646346,1646441,1655997,1665966,1668478,1674731,1824130,2060177,2094453,2128013,1709473,1710363,1710372,1762388,1769912,1771054,1780453,2335936,1787674,1782787,1791248,1795882,1952797,1798087,1807685,1828892,1843346,1859490,1857508,1858948,1877200,1865219,1866266,1868592,1878161,1889722,1895401,1930240,1955881,1895759,1897878,1925075,1900399,1903431,1906517,1918505,1924616,1927223,1930522,1941196,1947138,1950564,1956972,1956995,1966512,1990721,1992496,1996819,2012632,2013625,2321229,2376323,2071628,2096417,2032948,2056366,2045206,2045541,2046280,2049018,2058573,2076032,2067706,2097891,2082698,2083265,2084350,2085551,2088294,2089201,2093301,2094354,2106195,2100087,2105299,2111105,2115513,2135856,2142617,2142628,2143126,2148113,2148674,2148694,2148960,2150153,2171932,2151297,2151616,2154546,2164449,2170498,2178514,2179403,2184163,2188604,2194567,2202102,2214102,2218989,2244263,2244410,2253606,2256563,2278438,2305124,2365832,2385452,2497373,2504658,2618836]

df['alreadyConfirmed'] = df['consultID'].isin(phjPrevReadConsultations)

print(df)

In [ ]:
# Create a new dataframe that summarises proportion of flystrike consults per week
# ================================================================================
# Aggregate based on week_since_start
# -----------------------------------

# N.B. This graph plots the proportion of consultations found by the regex.
# You will need to adapt it to plot proportion of confirmed consultations.

dfGroupbyWeekDF = df.groupby('weeks_since_start').agg({'days_since_start': min,
                                                       'foundit': sum,
                                                       'consultID': 'count'})

# Reset the index - at the moment the weeks since start variable is the index
dfGroupbyWeekDF = dfGroupbyWeekDF.reset_index()

# Rename columns to something more meaningful
dfGroupbyWeekDF = dfGroupbyWeekDF.rename(columns = {'days_since_start': 'days_since_start',
                                                    'foundit': 'weeklyFlystrikeRegexMatches',
                                                    'consultID': 'weeklyTotalConsults'})

# Calculate weekly fireworks consultations per 1000 consultations
# ---------------------------------------------------------------
dfGroupbyWeekDF['weeklyFlystrikeRegexMatchesPer1000'] = (dfGroupbyWeekDF['weeklyFlystrikeRegexMatches']/dfGroupbyWeekDF['weeklyTotalConsults'])*1000

# Print just the top and bottom rows for 3 columns only
with pd.option_context('display.max_rows', 10, 'display.max_columns', 8):
    print(dfGroupbyWeekDF)

In [ ]:
# Plot proportion of flystrike consultations
dfGroupbyWeekDF.plot.line(x = 'weeks_since_start',
                             y = 'weeklyFlystrikeRegexMatchesPer1000')

### Export useful consultations to a CSV file, ready to be opened by Excel

In [ ]:
# Only retain consultations where a regex match was found
df = df.loc[df['foundit']==True,:]

In [ ]:
# When you run this box, you will be presented with an input box
# where you should enter the path and filename of the file you
# want to save.
#
# Enter the path and filename of where to save the csv file.
# On a Mac, the path will take the format:
#     ~/Documents/Research_project/myFile.csv
#
# On Windows, the path will take the form:
#     C:\Documents\Reserach_project\myFile.csv

pathToFile = input("Enter path and filename to save csv file: ")

df.to_csv(pathToFile)

print('Done')

The above code will produce a CSV file containing ONLY the consultations that match the regex together with some additional variables.